In [1]:
import pandas as pd
from sklearn.cluster import MeanShift

In [2]:
lines = []
data = []

with open('checkins.dat') as input_file:
    columns = list(map(lambda x: x.strip(), input_file.readline().strip().split('|')))
    n_params = len(columns)
    for line in input_file:
        items = list(map(lambda x: x.strip(), line.strip().split('|')))
        if len(items) != n_params:
            print(line)
        else:
            data.append(items)

---------+---------+----------+-------------------+-------------------+---------------------

(1021966 rows)





In [3]:
df = pd.DataFrame(data)
df.columns = columns
df = df[(df.latitude != '') & (df.longitude != '')]
df.head()

,id,user_id,venue_id,latitude,longitude,created_at
1,984222,15824,5222,38.8951118,-77.0363658,2012-04-21 17:43:47
3,984234,44652,5222,33.800745,-84.41052,2012-04-21 17:43:43
7,984291,105054,5222,45.5234515,-122.6762071,2012-04-21 17:39:22
9,984318,2146539,5222,40.764462,-111.904565,2012-04-21 17:35:46
10,984232,93870,380645,33.4483771,-112.0740373,2012-04-21 17:38:18


In [4]:
df.shape

(396634, 6)

In [5]:
data = df[['latitude', 'longitude']]

In [6]:
dataSample = data.sample(100000)

In [8]:
%%time
clusterisation = MeanShift(bandwidth=0.1)
clusterisation.fit(dataSample)

CPU times: user 2min 24s, sys: 1.03 s, total: 2min 25s
Wall time: 2min 25s


MeanShift(bandwidth=0.1, bin_seeding=False, cluster_all=True, min_bin_freq=1,
          n_jobs=None, seeds=None)

In [9]:
dataSample['Cluster'] = clusterisation.predict(dataSample)

In [14]:
clusterSize = dataSample.pivot_table(index = 'Cluster', aggfunc = 'count', values = 'latitude')
clusterSize.columns = ['size']

In [15]:
clusterCenters = pd.DataFrame(clusterisation.cluster_centers_)
clusterCenters.columns = ['cent_latitude', 'cent_longitude']

In [22]:
clusterDf = clusterCenters.join(clusterSize)
clusterDf = clusterDf[clusterDf['size']>15]

In [27]:
def getDistance(lat1, lon1, lat2, lon2):
    return ((lat1 - lat2)**2 + (lon1 - lon2)**2) ** 0.5

officeCoordinates = [
    (33.751277, -118.188740),
    (25.867736, -80.324116),
    (51.503016, -0.075479),
    (52.378894, 4.885084),
    (39.366487, 117.036146),
    (-33.868457, 151.205134)
]

def getMinDistanceToOffice(lat, lon):
    minDist = None
    for (of_lat, of_lon) in officeCoordinates:
        dist = getDistance(lat, lon, of_lat, of_lon)
        if (minDist is None) or (dist < minDist):
            minDist = dist
    return minDist

In [32]:
clusterDf['min_distance'] = list(map(getMinDistanceToOffice, clusterDf.cent_latitude, clusterDf.cent_longitude))

In [50]:
answerData = clusterDf.sort_values('min_distance')[:1]
answerLatitude = answerData['cent_latitude']
answerLongitude = answerData['cent_longitude']
ad = answerData[['cent_latitude', 'cent_longitude']].to_numpy()
str(ad[0, 0])

'-33.8658657862745'

In [51]:
with open('answer.txt', 'w') as f:
    f.write(str(ad[0, 0])+' '+str(ad[0, 1]))